In [2]:
import numpy as np
import json
import os
import re
import collections
from sklearn.datasets.base import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import codecs
import pickle
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText
import sys
from __future__ import division
from collections import OrderedDict
import pandas as pd
from sklearn.datasets import load_files
from collections import OrderedDict
import sys

In [3]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import KFold
from surprise import accuracy

In [4]:
def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [5]:
#Load Training Data change path name as per your directory
path = '/home/irlab/Documents/recsys/mpd/data'
filenames = os.listdir(path)
mapping = {}
playlist_prob = OrderedDict()

In [6]:
#Train FastText Model for finding similarity
#data.txt is created from the playlist names
#change location as per your 'files' folder location
def train_playlist_embedding(filenames):    
    #Change this as per you files folder location
    data = LineSentence('/home/irlab/files/data.txt')
    model = FastText(size=100)
    print('Building Vocab....')
    model.build_vocab(data)
    print('Training....')
    model.train(data, total_examples=model.corpus_count, epochs=model.iter)
    print(model)
    return model

In [7]:
model = train_playlist_embedding(filenames)

Building Vocab....
Training....


/home/irlab/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


FastText(vocab=9132, size=100, alpha=0.025)


In [8]:
#Find Similar Playlists with given name
def find_similar(playlist_name,model = model):
    names = []
    similar_playlists = model.wv.similar_by_word(playlist_name,20)
    similar_playlists = dict(similar_playlists)
    sim_playlist_name = [key for (key,value) in sorted(similar_playlists.items(),key=lambda x:x[1],reverse=True)]
    return sim_playlist_name

In [9]:
#Mapping of playlist name to filename
def load_filemapping():
    #Change this as per you files folder location
    fp = open('/home/irlab/files/mappings/track_filename.json')
    mpd_mapping = json.load(fp)
    return mpd_mapping

In [10]:
mpd_mapping = load_filemapping()

In [11]:
#Change this as per you files folder location
fp = open('/home/irlab/files/stats/album_stats.json')
album_map = json.load(fp)
album_stat = sum(album_map.values())

#Change this as per you files folder location
fp = open('/home/irlab/files/stats/artist_stats.json')
artist_map = json.load(fp)
artist_stat = sum(artist_map.values())

#Change this as per you files folder location
fp = open('/home/irlab/files/stats/tracks_stats.json')
track_map = json.load(fp)
track_stat = sum(track_map.values())

print(album_stat,artist_stat,track_stat)

(66346428, 66346428, 66346428)


In [12]:
#Generate weights for the tracks
def create_weighted_playlists(playlist_name,playlist_prob):
    try:
        similar_playlists = find_similar(playlist_name)
    except KeyError:
        temp = playlist_name.split()
        playlist_name = ''.join(temp)
        similar_playlists = find_similar(playlist_name)
        
    #print(similar_playlists)
    for similar_playlist in similar_playlists:
        if similar_playlist in mpd_mapping.keys():    
            #print(similar_playlist)
            name = mpd_mapping[similar_playlist]
            filename = os.sep.join((path,name))
            fp = open(filename,'r')
            playlist_file = json.load(fp)
            mpd_data = playlist_file['playlists']
            playlist_prob[similar_playlist] = {}
            for playlist in mpd_data:
                if normalize_name(playlist['name']) in similar_playlist:
                    for track in playlist['tracks']:
                        p1 = artist_map[track['artist_uri']]/artist_stat
                        p2 = album_map[track['album_uri']]/album_stat
                        p3 = track_map[track['track_uri']]/track_stat
                        playlist_prob[similar_playlist][track['track_uri']] = (p1+p2+p3)/(1+track['pos'])
    
    return playlist_prob

In [13]:
#Create Rating matrix
def create_matrix(playlist_prob):
    pd.DataFrame(data=playlist_prob,dtype=np.int64)
    df = pd.DataFrame(data=playlist_prob)
    df = df.fillna(0)
    df = df.T
    return df

In [14]:
#Load Data in surprise format
def load_data(tracks,playlists,ratings):
    ratings_dict = {'itemID': tracks,
                    'userID': playlists,
                    'rating': ratings}
    df1 = pd.DataFrame(ratings_dict)
    reader = Reader(rating_scale=(0,1))
    data = Dataset.load_from_df(df1[['userID','itemID','rating']],reader)
    #print(df1)
    return data

In [15]:
#Train using SVD and Gradient Descent
def train(data):
    kf = KFold(n_splits=10)
    algo = SVD(n_epochs=55,lr_all=0.018,reg_all=0.065)
    for trainset,testset in kf.split(data):
        algo.fit(trainset)
        try:
            prediction = algo.test(testset)
            print(accuracy.rmse(prediction,verbose=True))
        except UnicodeEncodeError:
            print()
    return algo

In [16]:
#Predict ratings for unknown tracks and filter them out
def test(playlist_name,tracks,algo,seed_tracks):
    tracks_pred = [n for n in tracks if n not in seed_tracks]
    Recommend = {}
    for track in tracks_pred:
        try:
            predict = algo.predict(playlist_name,track)
        except UnicodeEncodeError:
            predict = algo.predict('playlist_test',track)
            
        Recommend[track] = predict[3]
    recommend_list = [key for (key,value) in sorted(Recommend.items(),key=lambda x:x[1],reverse=True)]
    return recommend_list

In [17]:
#If we fall short of 500 tracks pick popular tracks
def random_tracks(recommend_list,seed_tracks):
    tracks = [key for (key,value) in sorted(track_map.items(),key=lambda x:x[1],reverse=True)]
    extend_tracks = [track for track in tracks if track not in recommend_list]
    extend_tracks = [track for track in extend_tracks if track not in seed_tracks]
    diff = 500 - len(recommend_list)
    print(len(extend_tracks[0:diff]))
    for name in extend_tracks[0:diff]:
        print(name)
        recommend_list.append(name)
    return recommend_list

In [18]:
#Change this path as per your system
path1 = '/home/irlab/Documents/recsys/challenge'
challenge_file = 'challenge_set.json'
filename = os.sep.join((path1,challenge_file))
fp = open(filename)
challenge_set = json.load(fp)
playlists = challenge_set['playlists']
dict_playlist = {}

In [19]:
import csv

In [72]:
#string = ['team_info','main','Dark_Horse_1','201711040@daiict.ac.in']

#with open('/home/irlab/Documents/recsys/mpd/codes/output/output2.csv','w') as csvfile:
#        filewriter = csv.writer(csvfile,delimiter = ',')
#        filewriter.writerow(string)

count=0
for playlist in playlists:
    list_temp = []
    seed_tracks = []
    recommend_list = []
    if 'name' in playlist.keys() and playlist['num_samples']==1:
        pid = playlist['pid']
        list_temp.insert(0,pid)
        count+=1
        print(count,playlist['name'])
        tracks = playlist['tracks']
        playlist_prob.clear()
        playlist_prob[normalize_name(playlist['name'])] = {}
        i=0
        for track in tracks:
            if track['artist_uri'] not in artist_map.keys():
                p1 = 1
            else:    
                p1 = artist_map[track['artist_uri']]/artist_stat
            if track['album_uri'] not in artist_map.keys():
                p2 = 1
            else:
                p2 = album_map[track['album_uri']]/album_stat
            if track['track_uri'] not in artist_map.keys():
                p3 = 1
            else:
                p3 = track_map[track['track_uri']]/track_stat
            seed_tracks.insert(i,track['track_uri'])
            playlist_prob[normalize_name(playlist['name'])][track['track_uri']] = (p1+p2+p3)/(1+track['pos']) 
            i+=1
            #print(seed_tracks)
        print('Finding Similar Playlists...')
        playlist_prob = create_weighted_playlists(normalize_name(playlist['name']),playlist_prob)
        print('Creating Rating matrix...')
        df = create_matrix(playlist_prob)
        column_names = list(df.columns)
        row_names = list(df.index)
        matrix = df.values
        row,col = np.nonzero(matrix)
        row = row.tolist()
        col = col.tolist()
        n_tracks = [column_names[i] for i in col]
        playlists = [row_names[i] for i in row]
        ratings = [matrix[r][c] for (r,c) in zip(row,col)]
        print('Loading data for training')
        data = load_data(n_tracks,playlists,ratings)
        print('Training....')
        algo = train(data)
        print('Predicting....')
        recommend_list = test(normalize_name(playlist['name']),n_tracks,algo,seed_tracks)
        print('Playlist Size:',len(recommend_list))
        if len(recommend_list[0:500])<500:
            print('Picking popular tracks...')
            recommend_list = random_tracks(recommend_list,seed_tracks)
            print('New Playlist Size:',len(recommend_list))
        list_temp.extend(recommend_list[0:500])
        print('Writing to file.....')
        #Change this path as per your system
        with open('/home/irlab/Documents/recsys/mpd/codes/output/final_submission.csv','a') as csvfile1:
            filewriter = csv.writer(csvfile1,delimiter = ',')
            filewriter.writerow(list_temp)

(1, u'Comedy')
(2, u'Reggae')
(3, u'Cover Songs ')
(4, u'banda')
(5, u'soft.')
(6, u'hozier')
(7, u'\U0001f634\U0001f634')
(8, u'reggae')
(9, u'Soul Food')
(10, u'CARS')
(11, u'Praise')
(12, u'Pep')
(13, u'\u2764\ufe0f\U0001f49b\U0001f49a\U0001f499\U0001f49c')
(14, u'swing')
(15, u'Better songs')
(16, u'Reggae')
(17, u'BMW')
(18, u'Get Pumped')
(19, u'It')
(20, u'Hot fire')
(21, u'chillax')
(22, u'Main')
(23, u'Famous')
(24, u'SleepTime ')
(25, u'two')
(26, u"Sam's Playlist")
(27, u'Fall')
(28, u'June 2016')
(29, u'Work Playlist!!!')
(30, u'Feb 2017')
(31, u'idk')
(32, u'Feelz')
(33, u'truth')
(34, u'30')
(35, u'Disney')
(36, u'shower jams')
(37, u'Future Disco')
(38, u'Fitness')
(39, u'Slow')
(40, u'17')
(41, u'Ride out')
(42, u'prblms')
(43, u'hyphy')
(44, u'Reggae ')
(45, u'Powerful')
(46, u'Down')
(47, u'JAMS.')
(48, u'Recently Added')
(49, u'April')
(50, u'Jammy Jams')
(51, u'beats')
(52, u'Tunes')
(53, u'running')
(54, u'Things')
(55, u'28')
(56, u'Summer')
(57, u'music')
(58, u'

RMSE: 0.0013
0.0012692658161589394
RMSE: 0.0155
0.0155433893456837
RMSE: 0.0022
0.0021682058263120282
RMSE: 0.0015
0.0015121719763533132
RMSE: 0.0013
0.001278284346990862
RMSE: 0.0018
0.0017565595647400596
RMSE: 0.0011
0.0011492657261634137
RMSE: 0.0075
0.007542888916989244
RMSE: 0.0898
0.08975388239383014
Predicting....
('Playlist Size:', 3640)
Writing to file.....
(91, u'SAMple')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0054
0.005362400677155382
RMSE: 0.0038
0.0037811217674423004
RMSE: 0.0022
0.002223843089828338
RMSE: 0.1776
0.17756098841602216
RMSE: 0.0024
0.0024193107295584428
RMSE: 0.0075
0.0075189834828677665
RMSE: 0.0010
0.0010189547926576761
RMSE: 0.0055
0.005493018914478555
RMSE: 0.0011
0.0010680858484173488
RMSE: 0.0014
0.0014295343009747936
Predicting....
('Playlist Size:', 1215)
Writing to file.....
(92, u'Disney')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE:

RMSE: 0.0017
0.0016799221883229364
RMSE: 0.0071
0.007137153641246788
RMSE: 0.0015
0.001514619222771613
RMSE: 0.0013
0.0013071079171222085
RMSE: 0.0021
0.002087890878817943
RMSE: 0.0017
0.0017096008696007392
RMSE: 0.1678
0.1678345937761465
RMSE: 0.0020
0.0019856116055511313
RMSE: 0.0017
0.0017354577118877965
RMSE: 0.0020
0.0020184592224175833
Predicting....
('Playlist Size:', 1350)
Writing to file.....
(107, u'heavy')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0009
0.0009235141327287964
RMSE: 0.0017
0.0017035678354647757
RMSE: 0.1280
0.1280458942368687
RMSE: 0.0007
0.0006576147787670765
RMSE: 0.0052
0.005180870074682434
RMSE: 0.0037
0.003661420951466315
RMSE: 0.0009
0.0009026867157097978
RMSE: 0.0007
0.0006709397246439645
RMSE: 0.0024
0.002394564407157218
RMSE: 0.0017
0.0016504912244368296
Predicting....
('Playlist Size:', 2354)
Writing to file.....
(108, u'Wish You Were Here')
Finding Similar Playlists...
Creating Rating matrix

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0005
0.0004582415638115549
RMSE: 0.0014
0.0014080233500063232
RMSE: 0.0032
0.003193694856272758
RMSE: 0.0010
0.0010392033470727495
RMSE: 0.0010
0.0010060045010170072
RMSE: 0.0011
0.0010684476061707977
RMSE: 0.0019
0.0018534067460144544
RMSE: 0.0036
0.00364565919031082
RMSE: 0.1586
0.15864427281254243
RMSE: 0.0016
0.0015517576469607141
Predicting....
('Playlist Size:', 1559)
Writing to file.....
(123, u'friends')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0007
0.000663906891190919
RMSE: 0.0010
0.00101771121295083
RMSE: 0.0011
0.0010967606161337961
RMSE: 0.0012
0.0011525853968469793
RMSE: 0.0020
0.002020629012453462
RMSE: 0.0012
0.001176649992370786
RMSE: 0.0019
0.0018545241913041253
RMSE: 0.1577
0.15767291230659414
RMSE: 0.0011
0.0011136542927286206
RMSE: 0.0014
0.001408730446165289
Predicting....
('Playlist Size:', 1582)
Writing to file.....
(124, u'all th

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0017
0.0017494214769133176
RMSE: 0.0029
0.0028703013296431023
RMSE: 0.0015
0.001450544490603884
RMSE: 0.0029
0.002900354592177897
RMSE: 0.0028
0.002809556426941963
RMSE: 0.0012
0.0011934571503660113
RMSE: 0.0014
0.0014311720336046478
RMSE: 0.0021
0.002096103824606543
RMSE: 0.0018
0.0017966302522533285
RMSE: 0.1032
0.10317564640940995
Predicting....
('Playlist Size:', 2955)
Writing to file.....
(139, u'Green')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0010
0.001017125900497287
RMSE: 0.1508
0.15075507568090787
RMSE: 0.0019
0.0019079389879946076
RMSE: 0.0018
0.0018365598055933299
RMSE: 0.0016
0.0015739083970823973
RMSE: 0.0016
0.0016170227403327381
RMSE: 0.0039
0.0038822548876858228
RMSE: 0.0020
0.001955313893881089
RMSE: 0.0023
0.002274649001431694
RMSE: 0.0026
0.00259280017014986
Predicting....
('Playlist Size:', 1561)
Writing to file.....
(140, u'Car Play

...

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0028
0.00276858974790917
RMSE: 0.1562
0.15619446331484804
RMSE: 0.0018
0.0017524985027431582
RMSE: 0.0027
0.002747302949620159
RMSE: 0.0014
0.001428089660612337
RMSE: 0.0017
0.0017149909376277458
RMSE: 0.0017
0.001687933765515249
RMSE: 0.0016
0.0016423176806530354
RMSE: 0.0014
0.0013504411610284445
RMSE: 0.0008
0.000844853445187214
Predicting....
('Playlist Size:', 1574)
Writing to file.....
(926, u'Pump up')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0089
0.008874007259689664
RMSE: 0.0016
0.0016348397765332744
RMSE: 0.0077
0.007694190419503523
RMSE: 0.0028
0.0028313626137560187
RMSE: 0.1314
0.1313725705371182
RMSE: 0.0030
0.002957504375386286
RMSE: 0.0081
0.008122583908292005
RMSE: 0.0079
0.007872812449191602
RMSE: 0.0024
0.0024008815079156636
RMSE: 0.0080
0.008002586420867807
Predicting....
('Playlist Size:', 2006)
Writing to file.....
(927, u'You Go Gir

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0030
0.0029793401589253386
RMSE: 0.0017
0.0016513002618511693
RMSE: 0.0007
0.0007426507918542096
RMSE: 0.0017
0.001692784093514845
RMSE: 0.0022
0.0022085971260651393
RMSE: 0.0009
0.0008577830970401999
RMSE: 0.0011
0.0011376259598861536
RMSE: 0.0011
0.0010692514297461022
RMSE: 0.0013
0.0013242851822543565
RMSE: 0.1145
0.11452034108597249
Predicting....
('Playlist Size:', 2754)
Writing to file.....
(942, u'Love Songs')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0043
0.004333138427390026
RMSE: 0.0038
0.003774618428091082
RMSE: 0.0053
0.005276888218337813
RMSE: 0.0018
0.001760566826125936
RMSE: 0.0038
0.003768653525585513
RMSE: 0.1577
0.15766140101679887
RMSE: 0.0030
0.00297193504586786
RMSE: 0.0060
0.005957077729596303
RMSE: 0.0030
0.003046325937876657
RMSE: 0.0006
0.0005789365184149522
Predicting....
('Playlist Size:', 1512)
Writing to file.....
(943, u'get 

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0031
0.0030724147747316715
RMSE: 0.0013
0.0012535491125530984
RMSE: 0.0008
0.0008394196408902714
RMSE: 0.0017
0.001710913909091387
RMSE: 0.0008
0.0007840745869391024
RMSE: 0.0909
0.09094422278978032
RMSE: 0.0010
0.0009971133366102494
RMSE: 0.0008
0.0008465025261180153
RMSE: 0.0011
0.0011284041126476672
RMSE: 0.0020
0.002016445962604458
Predicting....
('Playlist Size:', 3880)
Writing to file.....
(958, u'Sleep Music ')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0011
0.0011429115038000433
RMSE: 0.0057
0.005708574389889119
RMSE: 0.1023
0.1023349994089359
RMSE: 0.0051
0.005102016509501612
RMSE: 0.0014
0.0014269241559903166
RMSE: 0.0017
0.001659699946502011
RMSE: 0.0019
0.0019215967368411402
RMSE: 0.0054
0.005355557007968117
RMSE: 0.0057
0.005680291684365405
RMSE: 0.0039
0.0038784081422881074
Predicting....
('Playlist Size:', 3007)
Writing to file.....
(959, u'

Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0051
0.00506014689822878
RMSE: 0.1358
0.1358284572737711
RMSE: 0.0021
0.0020860276711719412
RMSE: 0.0019
0.0018590017388220308
RMSE: 0.0027
0.0026739180669687083
RMSE: 0.0021
0.0020732608963021527
RMSE: 0.0021
0.0021193358295430336
RMSE: 0.0017
0.0016925659364237409
RMSE: 0.0019
0.001948545661418657
RMSE: 0.0043
0.004300126906268159
Predicting....
('Playlist Size:', 1904)
Writing to file.....
(974, u'MEME SONGS')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0011
0.0011443968923192877
RMSE: 0.0026
0.00262880949756176
RMSE: 0.0011
0.0011043294545063677
RMSE: 0.0022
0.0022212607555381373
RMSE: 0.1656
0.16556930145672863
RMSE: 0.0015
0.001497263673666827
RMSE: 0.0026
0.002622765933643634
RMSE: 0.0022
0.002181071042117596
RMSE: 0.0029
0.00285904524092845
RMSE: 0.0021
0.0020920303419680215
Predicting....
('Playlist Size:', 1393)
Writin

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0023
0.0022813441495963403
RMSE: 0.0025
0.0024935841180296348
RMSE: 0.0021
0.0020647559642358944
RMSE: 0.1349
0.13486979674720806
RMSE: 0.0009
0.0009424627099392368
RMSE: 0.0020
0.0019619740652104477
RMSE: 0.0014
0.0013928868928724876
RMSE: 0.0012
0.001185890158219989
RMSE: 0.0029
0.0028899330894446174
RMSE: 0.0018
0.0018118008412931193
Predicting....
('Playlist Size:', 1725)
Writing to file.....
(990, u'November Playlist')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0030
0.0030111601449585314
RMSE: 0.0067
0.006651472821151954
RMSE: 0.0026
0.0025955712990563696
RMSE: 0.0013
0.0012635639172412468
RMSE: 0.0015
0.0014897427155452425
RMSE: 0.0025
0.0024883147613039835
RMSE: 0.1206
0.12060779993803186
RMSE: 0.0020
0.0020007572104059723
RMSE: 0.0009
0.0009415401120910958
RMSE: 0.0016
0.0015862560975809074
Predicting....
('Playlist Size:', 2405)
Writing to file...